# Alignment of EM sections

This notebook shows how to use SOFIMA to elastically align ultrathin sections of brain tissue acquired with an electron microscope. We will be using a 5000x5000x200 subset of a SBEM dataset of zebrafish brain acquired in Rainer Friedrich's lab at the FMI. You can browse the [unaligned data](https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B1.1e-8%2C%22m%22%5D%2C%22y%22:%5B1.1e-8%2C%22m%22%5D%2C%22z%22:%5B2.5e-8%2C%22m%22%5D%7D%2C%22position%22:%5B2359.354248046875%2C2653.44775390625%2C99.5%5D%2C%22crossSectionScale%22:3.5180938094527145%2C%22projectionScale%22:8192%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://sofima-sample-data/fmi-friedrich-dp/subvol_5800_5500_6250%22%2C%22tab%22:%22source%22%2C%22name%22:%22subvol_5800_5500_6250%22%7D%5D%2C%22selectedLayer%22:%7B%22visible%22:true%2C%22layer%22:%22subvol_5800_5500_6250%22%7D%2C%22layout%22:%224panel%22%7D) in Neuroglancer.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorstore
import tensorstore as ts

In [ ]:
storage_dir = "/content/drive/MyDrive/batch_004_4096_1_ts"  # 这里替换成你本地存储路径

In [3]:
# unaligned_1x = ts.open({
#     'driver': 'neuroglancer_precomputed',
#     'kvstore': 'gs://sofima-sample-data/fmi-friedrich-dp/subvol_5800_5500_6250',
#     "scale_metadata": {"resolution": [11, 11, 25]},
#     "context": { "cache_pool": {"total_bytes_limit": 1_000_000_000},}
#     }).result()

unaligned_1x = ts.open({
    'driver': 'neuroglancer_precomputed',  # 使用 Neuroglancer Precomputed 驱动
    'kvstore': f'file://{storage_dir}',  # 使用本地路径
    'scale_metadata': {"resolution": [11, 11, 25]},  # 设置 s0 层级的分辨率
    'context': {'cache_pool': {'total_bytes_limit': 1_000_000_000}},  # 设置缓存池
}).result()



ValueError: Error parsing object member "kvstore": Parsing spec from url: "/content/drive/MyDrive/batch_004_4096_1_ts": URL scheme must be specified in "/content/drive/MyDrive/batch_004_4096_1_ts" [source locations='tensorstore/kvstore/url_registry.cc:163\ntensorstore/kvstore/url_registry.cc:163\ntensorstore/kvstore/spec.cc:93\ntensorstore/internal/json_binding/json_binding.h:873\ntensorstore/internal/json_binding/json_binding.h:829']

In [ ]:
# Precomputed volume, 2x downsampled resolution. Downsampling was done with area averaging.
# unaligned_2x = ts.open({
#     'driver': 'neuroglancer_precomputed',
#     'kvstore': 'gs://sofima-sample-data/fmi-friedrich-dp/subvol_5800_5500_6250',
#     "scale_metadata": {"resolution": [22, 22, 25]},
#     "context": { "cache_pool": {"total_bytes_limit": 1_000_000_000},}
#     }).result()

unaligned_2x = ts.open({
    'driver': 'neuroglancer_precomputed',  # 使用 Neuroglancer Precomputed 驱动
    'kvstore': f'file://{storage_dir}',  # 使用本地路径
    'scale_metadata': {"resolution": [22, 22, 25]},  # 设置 s1 层级的分辨率
    'context': {'cache_pool': {'total_bytes_limit': 1_000_000_000}},  # 设置缓存池
}).result()

In [ ]:
unaligned_1x.shape

In [ ]:
!pip install git+https://github.com/google-research/sofima

In [ ]:
from concurrent import futures
import time

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from connectomics.common import bounding_box
from sofima import flow_field
from sofima import flow_utils
from sofima import map_utils
from sofima import mesh
from sofima import warp
from tqdm.notebook import tqdm

In [ ]:
# Ensure that we're running this code on a GPU machine. If this fails and you're using
# Google Colab, use "Edit >> Notebook settings" and set s"Hardware accelerator" to "GPU".
assert jax.devices()[0].platform == 'gpu'

# Flow field estimation

First, we calculate the flow fields between the current section and the directly preceding section. Flow fields can also be computed between pairs of sections that are not directly adjacent. This is useful if sections are incomplete or missing, but is not something we have to worry about in this demo.

In a distributed environment, this step would be done with the `EstimateFlow` processor.


On a V100, the expected time for the flow calculation over a single 5000x5000 section with the settings below, is ~0.6s. The patch (160) and step (40) sizes are set to conservative values which work for most synaptic-resolution EM volumes (i.e. at an in-plane resolution of ~10 nm/px).


In [ ]:
# Both of the settings below are expressed in pixels.
patch_size = 160  # XY spatial context used for flow field estimation
stride = 40  # XY distance between centers of adjacent patches.

def _compute_flow(volume):
  mfc = flow_field.JAXMaskedXCorrWithStatsCalculator()
  flows = []
  prev = volume[..., 0, 0].T.read().result()

  fs = []
  with futures.ThreadPoolExecutor() as tpe:
    # Prefetch the next sections to memory so that we don't have to wait for them
    # to load when the GPU becomes available.
    for z in range(1, unaligned_1x.shape[2]):
      fs.append(tpe.submit(lambda z=z: volume[..., z, 0].T.read().result()))

    fs = fs[::-1]

    for z in tqdm(range(1, unaligned_1x.shape[2])):
      curr = fs.pop().result()

      # The batch size is a parameter which impacts the efficiency of the computation (but
      # not its result). It has to be large enough for the computation to fully utilize the
      # available GPU capacity, but small enough so that the batch fits in GPU RAM.
      flows.append(mfc.flow_field(prev, curr, (patch_size, patch_size),
                                  (stride, stride), batch_size=256))
      prev = curr

  return flows

Compute flows at native and 2x reduced in-plane resolution. The lower resolution flow has reduced precision, but is helpful for providing approximate flow vectors in places where the full-resolution flow might be impossible to estimate, e.g. in the interior of cell bodies or blood vessels.


In [ ]:
flows1x = np.array(_compute_flow(unaligned_1x))
flows2x = np.array(_compute_flow(unaligned_2x))

The flow fields generated in the previous step are 4-channel arrays, where the first two channels store the XY components of the flow vector, and the two remaining channels are measures of estimation quality (see `sofima.flow_field._batched_peaks` for more info).

In [ ]:
# Convert to [channels, z, y, x].
flows2x = np.transpose(flows2x, [1, 0, 2, 3])
flows1x = np.transpose(flows1x, [1, 0, 2, 3])

# Pad to account for the edges of the images where there is insufficient context to estimate flow.
pad = patch_size // 2 // stride
flows1x = np.pad(flows1x, [[0, 0], [0, 0], [pad, pad], [pad, pad]], constant_values=np.nan)
flows2x = np.pad(flows2x, [[0, 0], [0, 0], [pad, pad], [pad, pad]], constant_values=np.nan)

We now remove uncertain flow estimates by replacing them with NaNs, and merge the two flow arrays into a single flow field at full resolution. In a distributed environment, this step would be done with the `ReconcileAndFilterFlows` processor.

In [ ]:
f1 = flow_utils.clean_flow(flows1x, min_peak_ratio=1.6, min_peak_sharpness=1.6, max_magnitude=80, max_deviation=20)
f2 = flow_utils.clean_flow(flows2x, min_peak_ratio=1.6, min_peak_sharpness=1.6, max_magnitude=80, max_deviation=20)

Plot the horizontal component of the flow vector, before (left) and after (right) filtering. While blobs indicate areas where uncertain flow estimates were removed.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(5, 2.5))
ax[0].imshow(flows1x[0, 14, ...], cmap=plt.cm.RdBu, vmin=-10, vmax=10)
ax[1].imshow(f1[0, 14, ...], cmap=plt.cm.RdBu, vmin=-10, vmax=10)

In [ ]:
from scipy import interpolate
f2_hires = np.zeros_like(f1)

scale = 0.5
oy, ox = np.ogrid[:f2.shape[-2], :f2.shape[-1]]
oy = oy.ravel() / scale
ox = ox.ravel() / scale

box1x = bounding_box.BoundingBox(start=(0, 0, 0), size=(f1.shape[-1], f1.shape[-2], 1))
box2x = bounding_box.BoundingBox(start=(0, 0, 0), size=(f2.shape[-1], f2.shape[-2], 1))

for z in tqdm(range(f2.shape[1])):
  # Upsample and scale spatial components.
  resampled = map_utils.resample_map(
      f2[:, z:z + 1, ...],  #
      box2x, box1x, 1 / scale, 1)
  f2_hires[:, z:z + 1, ...] = resampled / scale

In [ ]:
final_flow = flow_utils.reconcile_flows((f1, f2_hires), max_gradient=0, max_deviation=20, min_patch_size=400)

In [ ]:
# Plot (left to right): high res. flow, upsampled low res. flow, combined flow to use for alignment.
f, ax = plt.subplots(1, 3, figsize=(7.5, 2.5))
ax[0].imshow(f1[0, 14, ...], cmap=plt.cm.RdBu, vmin=-10, vmax=10)
ax[1].imshow(f2_hires[0, 14, ...], cmap=plt.cm.RdBu, vmin=-10, vmax=10)
ax[2].imshow(final_flow[0, 14, ...], cmap=plt.cm.RdBu, vmin=-10, vmax=10)

# Mesh optimization

We use an elastic mesh optimizer to find a configuration of the imagery that is compatible with the estimated flow field and preserves the original geometry as much as possible.

The optimization proceeds sequentially, section by section. In a distributed environment, this computation can be parallelized across the plane (by independently solving overlapping XY tiles), as well as split into blocks along the Z axis. This makes it possible to scale this process to arbitrarily large volumes. For simplicity, here we solve the complete stack in one go.

In [ ]:
config = mesh.IntegrationConfig(dt=0.001, gamma=0.0, k0=0.01, k=0.1, stride=(stride, stride), num_iters=1000,
                                max_iters=100000, stop_v_max=0.005, dt_max=1000, start_cap=0.01,
                                final_cap=10, prefer_orig_order=True)

In [ ]:
solved = [np.zeros_like(final_flow[:, 0:1, ...])]
origin = jnp.array([0., 0.])

for z in tqdm(range(0, final_flow.shape[1])):
  prev = map_utils.compose_maps_fast(final_flow[:, z:z+1, ...], origin, stride,
                                     solved[-1], origin, stride)
  x = np.zeros_like(solved[0])
  x, e_kin, num_steps = mesh.relax_mesh(x, prev, config)
  x = np.array(x)
  solved.append(x)

In [ ]:
solved = np.concatenate(solved, axis=1)

# Image warping

Image warping requires an inverse coordinate map, so compute that first. In a distributed environment, this can be done with the `InvertMap` processor.

In [ ]:
inv_map = map_utils.invert_map(solved, box1x, box1x, stride)

We are now ready to render the aligned subvolume. To reduce RAM usage, we render a 1000x1000 part for all 200 sections.

In a distributed environment, this can be done with the `WarpByMap` processor.

In [ ]:
warped = [np.transpose(unaligned_1x[1000:2000, 2000:3000, 0:1, 0].read().result(), [2, 1, 0])]

for z in tqdm(range(1, unaligned_1x.shape[2])):
  data_box = bounding_box.BoundingBox(start=(500, 1500, 0), size=(2000, 2000, 1))
  out_box = bounding_box.BoundingBox(start=(1000, 2000, 0), size=(1000, 1000, 1))

  data = np.transpose(unaligned_1x[data_box.start[0]:data_box.end[0],
                                   data_box.start[1]:data_box.end[1],
                                   z:z+1, 0:1].read().result(), [3, 2, 1, 0])
  warped.append(
      warp.warp_subvolume(data, data_box, inv_map[:, z:z+1, ...], box1x, stride, out_box, 'lanczos', parallelism=1)[0, ...])

In [ ]:
warped_xyz = np.transpose(np.concatenate(warped, axis=0), [2, 1, 0])

Render an XZ cross section to check the quality of the alignment visually.

In [ ]:
plt.figure(figsize=(7, 3))
plt.imshow(warped_xyz[:, 500, :].T, cmap=plt.cm.Greys_r, aspect=(1000 * 11) / (200 * 25))

The results can be inspected interactively in Neuroglancer as well.

In [ ]:
!pip install neuroglancer
import neuroglancer

dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[11, 11, 25])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
  s.dimensions = dimensions
  s.layers['em'] = neuroglancer.ImageLayer(source=neuroglancer.LocalVolume(warped_xyz, dimensions))

viewer

# Distributed mesh optimization

The serial mesh optimization process described above might be too slow for large volumes. Fortunately, mesh optimization can be parallelized by splitting the volume into overlapping blocks of sections, solving the blocks independently in parallel, and reconciling the solutions to produce a globally consistent mesh.

If we now represent every block as a virtual "section" and use solved state of the last section of every block as the "flow field", we can reuse the existing SOFIMA functionality to optimize a new mesh representing the blocks. This can then be combined with the in-block meshes to form a global solution.

To demonstrate how this works, we will now solve the same mesh as before independently for [0..50], [50..100], [100..150], [150..199]. Note that the blocks overlap by exactly one and that in a real setting, this would be done in parallel instead of in a loop.


In [ ]:
from connectomics.volume import subvolume
from sofima.processor import maps

In [ ]:
block1 = [np.zeros_like(final_flow[:, 0:1, ...])]
block2 = [np.zeros_like(final_flow[:, 0:1, ...])]
block3 = [np.zeros_like(final_flow[:, 0:1, ...])]
block4 = [np.zeros_like(final_flow[:, 0:1, ...])]

for blk, start, stop in ((block1, 0, 50), (block2, 50, 100), (block3, 100, 150), (block4, 150, final_flow.shape[1])):
  for z in tqdm(range(start, stop)):
    prev = map_utils.compose_maps_fast(final_flow[:, z:z+1, ...], origin, stride,
                                       blk[-1], origin, stride)
    x = np.zeros_like(blk[0])
    x, e_kin, num_steps = mesh.relax_mesh(x, prev, config)
    x = np.array(x)
    blk.append(x)

In [ ]:
block1 = np.concatenate(block1, axis=1)
block2 = np.concatenate(block2, axis=1)
block3 = np.concatenate(block3, axis=1)
block4 = np.concatenate(block4, axis=1)

The mesh for the last section of 'block1' tells us where the first section of 'block2' should be, if block2 was aligned to block1. This mesh data for the last section of every block can be stacked to form a 'cross-block' flow field. We will downsample it 2x in-plane as the block-wise mesh can be coarser.


In [ ]:
map_box = bounding_box.BoundingBox(start=(0, 0, 0), size=block1.shape[1:][::-1])
map2x_box = map_box.scale(0.5)
xblk_stride = stride * 2

In [ ]:
xblock_flow = np.stack([block1[:, -1, ...], block2[:, -1, ...],
                        block3[:, -1, ...], block4[:, -1, ...]], axis=1)
xblock_flow = map_utils.resample_map(xblock_flow, map_box, map2x_box, stride, xblk_stride)

With the flow field prepared, we are aready to optimize the block-wise mesh. To take into account that every mesh section represents a much thicker piece of physical tissue, we lower the value of $k_0$, making the inter-section forces weak relative to the intra-section ones.

In [ ]:
xblk_config = mesh.IntegrationConfig(dt=0.001, gamma=0.0, k0=0.001, k=0.1, stride=(xblk_stride, xblk_stride), num_iters=1000,
                                     max_iters=100000, stop_v_max=0.005, dt_max=1000, start_cap=0.01,
                                     final_cap=10, prefer_orig_order=True)

xblk = []
for z in tqdm(range(xblock_flow.shape[1])):
  if z == 0:
    prev = xblock_flow[:, z:z+1, ...]
  else:
    prev = map_utils.compose_maps_fast(xblock_flow[:, z:z+1, ...], origin, xblk_stride, xblk[-1], origin, xblk_stride)
  x = np.zeros_like(xblock_flow[:, 0:1, ...])
  x, e_kin, num_steps = mesh.relax_mesh(x, prev, xblk_config)
  x = np.array(x)
  xblk.append(x)

In [ ]:
xblk = np.concatenate(xblk, axis=1)

The cross-block mesh can now be combined with the per-block solutions to form a globally consistent mesh.

In [ ]:
main = np.concatenate([block1[:, :50], block2[:, :50], block3[:, :50], block4], axis=1)
last = np.zeros_like(main)
last[:, 50, ...] = block1[:, -1, ...]
last[:, 100, ...] = block2[:, -1, ...]
last[:, 150, ...] = block3[:, -1, ...]
last[:, 199, ...] = block4[:, -1, ...]

main_inv = map_utils.invert_map(main, map_box, map_box, stride)
last_inv = map_utils.invert_map(last, map_box, map_box, stride)
xblk_upsampled = map_utils.resample_map(xblk, map2x_box, map_box, stride * 2, stride)
xblk_inv = map_utils.invert_map(xblk_upsampled, map_box, map_box, stride)

In [ ]:
class ReconcileCrossBlockMaps(maps.ReconcileCrossBlockMaps):
  def _open_volume(self, path: str):
    if path == 'main_inv':
      return main_inv
    elif path == 'last_inv':
      return last_inv
    elif path == 'xblk':
      return xblk_upsampled
    elif path == 'xblk_inv':
      return xblk_inv
    else:
      raise ValueError(f'Unknown volume {path}')

config = maps.ReconcileCrossBlockMaps.Config(
    cross_block='xblk',
    cross_block_inv='xblk_inv',
    last_inv='last_inv',
    main_inv='main_inv',
    z_map={'50': 0, '100': 1, '150': 2, '199': 3},
    stride=stride,
    xy_overlap=0)
reconcile = ReconcileCrossBlockMaps(config)
reconcile.set_effective_subvol_and_overlap(map_box.size, (0, 0, 0))
main_box = bounding_box.BoundingBox(start=(0, 0, 0), size=main.shape[1:][::-1])
global_map = reconcile.process(subvolume.Subvolume(main, main_box))

We're now ready to render the aligned images...

In [ ]:
global_map_inv = map_utils.invert_map(global_map.data, map_box, map_box, stride)
warped2 = [np.transpose(unaligned_1x[1000:2000, 2000:3000, 0:1, 0].read().result(), [2, 1, 0])]

for z in tqdm(range(1, unaligned_1x.shape[2])):
  data_box = bounding_box.BoundingBox(start=(500, 1500, 0), size=(2000, 2000, 1))
  out_box = bounding_box.BoundingBox(start=(1000, 2000, 0), size=(1000, 1000, 1))

  data = np.transpose(unaligned_1x[data_box.start[0]:data_box.end[0],
                                   data_box.start[1]:data_box.end[1],
                                   z:z+1, 0:1].read().result(), [3, 2, 1, 0])
  warped2.append(
      warp.warp_subvolume(data, data_box, global_map_inv[:, z:z+1, ...], box1x, stride, out_box, 'lanczos', parallelism=1)[0, ...])

warped2_xyz = np.transpose(np.concatenate(warped2, axis=0), [2, 1, 0])

... and inspect them as a static image ...

In [ ]:
plt.figure(figsize=(7, 3))
plt.imshow(warped2_xyz[:, 500, :].T, cmap=plt.cm.Greys_r, aspect=(1000 * 11) / (200 * 25))

... or interactively.

In [ ]:
import neuroglancer

dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[11, 11, 25])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
  s.dimensions = dimensions
  s.layers['em'] = neuroglancer.ImageLayer(source=neuroglancer.LocalVolume(warped2_xyz, dimensions))

viewer